<a href="https://colab.research.google.com/github/devinbook/Tagalog-profanity-detection/blob/main/XLNeT_ProfanityDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers torch scikit-learn datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
!pip install datasets

In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import XLNetTokenizer, XLNetForSequenceClassification, TrainingArguments, Trainer
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


In [ ]:
# Load the dataset
df = pd.read_csv("/content/combined_features_with_custom_vocab_and_full_vocab (2).csv")  # Replace with your file path

# Rename target column to 'labels' (required for Hugging Face Trainer)
df = df.rename(columns={"target": "labels"})

# Inspect the dataset
print(df.head())

   aabot  aabutin  aadjust  aalaga  aalala  aalis  aanak  aanhin  aanounce  \
0    0.0      0.0      0.0     0.0     0.0    0.0    0.0     0.0       0.0   
1    0.0      0.0      0.0     0.0     0.0    0.0    0.0     0.0       0.0   
2    0.0      0.0      0.0     0.0     0.0    0.0    0.0     0.0       0.0   
3    0.0      0.0      0.0     0.0     0.0    0.0    0.0     0.0       0.0   
4    0.0      0.0      0.0     0.0     0.0    0.0    0.0     0.0       0.0   

   aantay  ...  zero  zoom  zuckerberg  num_chars  num_words  num_sentences  \
0     0.0  ...   0.0   0.0         0.0        113         19              0   
1     0.0  ...   0.0   0.0         0.0         64          9              0   
2     0.0  ...   0.0   0.0         0.0         79         16              0   
3     0.0  ...   0.0   0.0         0.0         43          8              0   
4     0.0  ...   0.0   0.0         0.0         81         14              0   

   contains_profanity                                   

In [ ]:
print(df['Text'].isnull().sum())

# Drop rows with missing Text values
df = df.dropna(subset=['Text'])

# Optionally reset the index
df = df.reset_index(drop=True)

# Convert the DataFrame to a Hugging Face Dataset
dataset = Dataset.from_pandas(df)

1


In [ ]:
# Split into train and test datasets
train_test_split = dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

# Load XLNet tokenizer
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')

# Tokenize the text in the train and test sets
def tokenize_function(example):
    return tokenizer(
        example["Text"],  # Ensure this references the correct column
        truncation=True,
        padding=True,
        max_length=128  # Adjust as necessary
    )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/798k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.38M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/760 [00:00<?, ?B/s]

In [ ]:
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)

tokenized_train.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
tokenized_test.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

Map:   0%|          | 0/13616 [00:00<?, ? examples/s]

Map:   0%|          | 0/3405 [00:00<?, ? examples/s]

In [ ]:
# Load XLNet model with a classification head (adjust for 4 classes: High, Mild, Moderate, Non-profane)
model = XLNetForSequenceClassification.from_pretrained('xlnet-base-cased', num_labels=4)

pytorch_model.bin:   0%|          | 0.00/467M [00:00<?, ?B/s]

Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',             # Directory to save the model checkpoints
    evaluation_strategy="epoch",        # Evaluate at the end of each epoch
    save_strategy="epoch",              # Save the model after each epoch
    learning_rate=2e-5,                 # Learning rate
    per_device_train_batch_size=16,     # Batch size for training
    per_device_eval_batch_size=16,      # Batch size for evaluation
    num_train_epochs=5,                 # Number of epochs
    weight_decay=0.01,                  # Weight decay for regularization
    logging_dir='./logs',               # Directory to save logs
    logging_steps=10,                   # Log every 10 steps
    load_best_model_at_end=True,        # Load the best model at the end of training
    metric_for_best_model="f1",         # Metric to determine the best model
)

# Define metrics computation function
def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-9-1d889488fc0a>:31: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.112600,0.075939,0.986490,0.986605,0.986490,0.986482
2,0.001200,0.057813,0.989427,0.989530,0.989427,0.989396
3,0.006300,0.040841,0.994126,0.994139,0.994126,0.994119
4,0.053200,0.033615,0.994714,0.994745,0.994714,0.994709
5,0.003600,0.032986,0.995301,0.995320,0.995301,0.995295


TrainOutput(global_step=4255, training_loss=0.20393337642830894, metrics={'train_runtime': 2387.2322, 'train_samples_per_second': 28.518, 'train_steps_per_second': 1.782, 'total_flos': 4848742772981760.0, 'train_loss': 0.20393337642830894, 'epoch': 5.0})

In [ ]:
# Evaluate the model on the test dataset
evaluation_results = trainer.evaluate()

# Print the results
print(evaluation_results)

{'eval_loss': 0.03298572823405266, 'eval_accuracy': 0.9953010279001469, 'eval_precision': 0.9953203067845589, 'eval_recall': 0.9953010279001469, 'eval_f1': 0.9952953162848783, 'eval_runtime': 39.4309, 'eval_samples_per_second': 86.354, 'eval_steps_per_second': 5.402, 'epoch': 5.0}


In [ ]:
# Save the model and tokenizer
model.save_pretrained('./XLnet_profanitydetection')
tokenizer.save_pretrained('./XLnet_profanitydetection')

print("Model and tokenizer saved as 'XLnet_profanitydetection'")


Model and tokenizer saved as 'XLnet_profanitydetection'


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Define the save path in Google Drive
save_path = '/content/drive/My Drive/Saved_Models/XLnet_profanitydetection'

# Save the model and tokenizer
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print(f"Model and tokenizer saved to: {save_path}")

Model and tokenizer saved to: /content/drive/My Drive/Saved_Models/XLnet_profanitydetection
